In [10]:
import pandas as pd
from dhis2 import Api

from src.main.common import FileUtil
from src.main.application.income import *
from src.main.application.service import *
from src.infrastructure.forms import *
from src.infrastructure.adapters import *

In [2]:
credentials  = FileUtil.load_credentias()
username = credentials['username']
password = credentials['password']

url = 'https://dhis-ao.icap.columbia.edu'
api = Api(url, username, password)

In [3]:
# Get the list of supported facilities
org_unit = 'UnCWi0cmwXy'

start_period = '2024-01-01'
end_period = '2024-03-31'

In [4]:
#get all patient enrollments
patients = api.get('tracker/enrollments', params={'orgUnit':org_unit, 'skipPaging':'true', 'program': CARE_AND_TREATMENT, 'fields':'{,enrollment, enrolledAt, trackedEntity, program, status,}'})
patients = patients.json()['instances']

for patient in patients:
    patient_id = patient['trackedEntity']

    patient_demographics = PatientDemographicForm(patient_id, org_unit, api)
    patient_demographics.add_demographics(patient)

    patient_consultation= PatientConsultationForm(patient_id, org_unit, api)
    patient_consultation.add_first_consultation(patient)

    patient_pharmacy_form =  PatientPharmacyForm(patient_id, org_unit, api)
    patient_pharmacy_form.add_last_pharmacy(patient)

    patient_laboratory_from =  PatientLaboratoryForm(patient_id, org_unit, api)
    patient_laboratory_from.add_laboratory(patient)

    patient_outcome_form =  PatientOutcomeForm(patient_id, org_unit, api)
    patient_outcome_form.add_final_outcome(patient)

In [11]:
tx_curr_service = ComputeTxCurrService()
tx_curr_service.compute(patients, end_period)

tx_new = ComputeTxNewService(tx_curr_service)
tx_new.compute(patients,start_period,end_period)

tx_ml = ComputeTxMlService()
tx_ml.compute(patients, start_period, end_period)

laboratory_port = LaboratoryAdapter(patient_laboratory_from)
tx_pvls_denominator = ComputeTxPvlsDenominatorService(tx_curr_service, laboratory_port)

In [13]:
data = pd.json_normalize(patients)
data

,enrollment,trackedEntity,program,status,enrolledAt,patientIdentifier,patientAge,patientName,patientSex,artStartDate,firstConsultationDate,pickupQuantity,lastPickupDate,nextPickupDate,txCurr,txNew,txML,lastCD4
0,vpIb6ybxHPt,nOtmbnCaUMX,YKs1kXOLb0u,ACTIVE,2024-03-15T00:00:00.000,1609800880/2012T/00042,1988-03-27,Plenaria Nandjembo,Feminino,2012-03-16,2012-03-16,180,2024-03-15,2024-09-11,True,False,False,NaN
1,SCl2bte4YaG,X8NWMVxpxQ7,YKs1kXOLb0u,ACTIVE,2024-03-13T00:00:00.000,1609800880/2010T/00023,1966-03-27,Indilene Venoshipunge,Feminino,2010-02-28,2010-02-28,180,2024-03-13,2024-09-09,True,False,False,NaN
2,PNWWnTtoK9h,deYsA7onXqd,YKs1kXOLb0u,ACTIVE,2024-03-15T00:00:00.000,1609800880/2022T/00006,1994-03-27,Basilia Teresa,Feminino,2022-01-13,2022-01-13,180,2024-03-15,2024-09-11,True,False,False,NaN
3,rCfxfpY437c,qIvHoLs556q,YKs1kXOLb0u,ACTIVE,2024-03-15T00:00:00.000,1609800880/2022T/00068,1987-03-27,Plenaria Shefeni,Feminino,2006-08-08,2006-08-11,180,2024-03-15,2024-09-11,True,False,False,NaN
4,CLur1H0LYtT,eMcqYgaDIis,YKs1kXOLb0u,ACTIVE,2024-03-15T00:00:00.000,1609800880/2022T/00072,1998-03-27,Utefania Shituleipo,Feminino,2022-07-27,2022-07-27,90,2024-03-15,2024-06-13,True,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,l9uZ8BpkxpS,jSgyE50LxE6,YKs1kXOLb0u,ACTIVE,2024-03-13T00:00:00.000,1609800880/2009T/00011,1985-05-25,Adelina R.M. Luciano,Feminino,2008-12-29,2008-12-29,180,2024-03-13,2024-09-09,True,False,False,NaN
98,aPrIcv0SRtq,JyDtzx2Yhu9,YKs1kXOLb0u,ACTIVE,2024-03-13T00:00:00.000,1609800880/2021T/00161,1979-10-08,Emanuel Nd. Hipulavali,Masculino,2021-12-15,2021-12-15,180,2024-03-13,2024-09-09,True,False,False,NaN
99,XOHRk3ZXOrC,LXS92leAle7,YKs1kXOLb0u,ACTIVE,2024-03-13T00:00:00.000,1609800880/2021T/00016,1992-09-21,Isata Da Conceiçao,Feminino,2021-02-03,2021-02-03,180,2024-03-13,2024-09-09,True,False,False,NaN
100,xzhGjf2Jegb,g0jjX5MOLUG,YKs1kXOLb0u,ACTIVE,2024-03-13T00:00:00.000,1609800880/2022T/00064,1985-03-05,Monica Haufiku,Feminino,2020-03-20,2020-03-20,180,2024-03-12,2024-09-08,True,False,False,NaN
